## Import DOCX file

In [1]:
from docx import Document

In [2]:
document = Document(r"../res/test-swiss.docx")
document

In [3]:
for table in document.tables:
    for row in table.rows:
        for cell in row.cells:
            print(cell.text)

COUNTRY: SWITZERLAND
PROSPECTS
PROSPECTS
PROSPECTS
PROSPECTS
PROSPECTS
PROSPECTS
GROUP 
ACTIVITY
CLIENT TYPE
INTERACTION MODE
OFFSHORE SCENARIO
ONSHORE SCENARIO
Social Contact 

Social interactions

Retail
Pro-actively
YES
YES
Social Contact 

Social interactions

Retail
Upon RS
YES       
YES
Social Contact 

Social interactions

Professionals

Pro-actively
YES
YES
Social Contact 

Social interactions

Professionals

Upon RS
YES
YES
Social Contact 

Provide contact details
Retail
Pro-actively 
YES
YES
Social Contact 

Provide contact details
Retail
Upon RS 
YES
YES
Social Contact 

Provide contact details
Professionals

Pro-actively 
YES
YES
Social Contact 

Provide contact details
Professionals

Upon RS 
YES
YES
EXISTING CLIENTS
EXISTING CLIENTS
EXISTING CLIENTS
EXISTING CLIENTS
EXISTING CLIENTS
EXISTING CLIENTS
GROUP 
ACTIVITY
CLIENT TYPE
INTERACTION MODE
OFFSHORE SCENARIO
ONSHORE SCENARIO
Social Contact 

Social interactions

Retail
Pro-actively
YES
YES
Social Contact 

Social inte

SUBACTIVITY 
CLIENT TYPE
INTERACTION MODE
OFFSHORE SCENARIO
ONSHORE SCENARIO
Preparatory administrative activities 

Provide forms and contractual documents
Account opening contracts including related forms
Retail
Pro-actively
NO
NO
Preparatory administrative activities 

Provide forms and contractual documents
Account opening contracts including related forms
Retail
Upon RS
YES with a fiduciary company and subject to annex 1
NO without a fiduciary company (exceptional cases to be assessed)
NO
Preparatory administrative activities 

Provide forms and contractual documents
Account opening contracts including related forms
Professionals
Pro-actively
NO
NO
Preparatory administrative activities 

Provide forms and contractual documents
Account opening contracts including related forms
Professionals
Upon RS
YES with a fiduciary company and subject to annex 1
NO without a fiduciary company (exceptional cases to be assessed)
NO
Preparatory administrative activities 

Provide forms and contrac

Credit card agreements including related forms
Professionals
Pro-actively
YES
NO
Preparatory administrative activities 

Provide forms and contractual documents
Credit card agreements including related forms
Professionals
Upon RS
YES
NO
Preparatory administrative activities 

Provide forms and contractual documents
Other contracts and forms (securities lending, wealth management services and other types of contract)
Retail
Pro-actively
YES
NO
Preparatory administrative activities 

Provide forms and contractual documents
Other contracts and forms (securities lending, wealth management services and other types of contract)
Retail
Upon RS
YES
NO
Preparatory administrative activities 

Provide forms and contractual documents
Other contracts and forms (securities lending, wealth management services and other types of contract)
Professionals
Pro-actively
YES
NO
Preparatory administrative activities 

Provide forms and contractual documents
Other contracts and forms (securities lending, weal


Marketing of financial instruments
with advisory mandate
with RSSI 

Professionals
Upon RS 
YES
NO 
Performance of activities 

Marketing of financial instruments
without advisory mandate

Retail
Pro-actively 
NO 
NO 
Performance of activities 

Marketing of financial instruments
without advisory mandate

Retail
Upon RS 
YES
NO
Performance of activities 

Marketing of financial instruments
without advisory mandate

Professionals
Pro-actively 
NO 
NO 
Performance of activities 

Marketing of financial instruments
without advisory mandate

Professionals
Upon RS 
YES 
YES 
Performance of activities 

Generic Market Views
with advisory mandate
with RSSI 

Retail
Pro-actively 
YES 
YES 
Performance of activities 

Generic Market Views
with advisory mandate
with RSSI 

Retail
Upon RS 
YES
YES
Performance of activities 

Generic Market Views
with advisory mandate
with RSSI 

Professionals
Pro-actively 
YES
YES
Performance of activities 

Generic Market Views
with advisory mandate
with RSSI 


## Make dataframes from tables

In [4]:
import pandas as pd
import io
import csv
from docx import Document

def read_docx_tables(filename, tab_id=None, **kwargs):
    """
    parse table(s) from a Word Document (.docx) into Pandas DataFrame(s)

    Parameters:
        filename:   file name of a Word Document

        tab_id:     parse a single table with the index: [tab_id] (counting from 0).
                    When [None] - return a list of DataFrames (parse all tables)

        kwargs:     arguments to pass to `pd.read_csv()` function

    Return: a single DataFrame if tab_id != None or a list of DataFrames otherwise
    """
    def read_docx_tab(tab, **kwargs):
        vf = io.StringIO()
        writer = csv.writer(vf)
        for row in tab.rows:
            writer.writerow(cell.text for cell in row.cells)
        vf.seek(0)
        return pd.read_csv(vf, **kwargs)

    doc = Document(filename)
    if tab_id is None:
        return [read_docx_tab(tab, **kwargs) for tab in doc.tables]
    else:
        try:
            return read_docx_tab(doc.tables[tab_id], **kwargs)
        except IndexError:
            print('Error: specified [tab_id]: {}  does not exist.'.format(tab_id))
            raise

In [5]:
dfs = read_docx_tables(r"../res/test-swiss.docx")

## Assemble in app

In [7]:
dfs = [df for df in dfs if df.shape != (0, 1)]

len(dfs)

11

In [8]:
for idx, df in enumerate(dfs):
    precondition = True if df.columns[0].startswith('PRE-CONDITION') else False
    columns = list(df.iloc[0,]) if not precondition else list(df.iloc[1,])
    connection_type = df.columns[0] if not precondition else df.iloc[0,0]

    df.columns = columns
    df.insert(0, "CONNECTION TYPE", connection_type)
    df.drop(df.head(1).index if not precondition else df.head(2).index, inplace=True)

In [9]:
for idx, df in enumerate(dfs):
    id_vars = [c for c in df.columns if not c.endswith('SCENARIO')]
    dfs[idx] = df.melt(id_vars=id_vars)
    dfs[idx].rename(columns={'value': 'OUTCOME', 'variable': 'SCENARIO'}, inplace=True)

In [10]:
dfs[0].head()

,CONNECTION TYPE,GROUP,ACTIVITY,CLIENT TYPE,INTERACTION MODE,SCENARIO,OUTCOME
0,PROSPECTS,Social Contact \n,Social interactions\n,Retail,Pro-actively,OFFSHORE SCENARIO,YES
1,PROSPECTS,Social Contact \n,Social interactions\n,Retail,Upon RS,OFFSHORE SCENARIO,YES
2,PROSPECTS,Social Contact \n,Social interactions\n,Professionals\n,Pro-actively,OFFSHORE SCENARIO,YES
3,PROSPECTS,Social Contact \n,Social interactions\n,Professionals\n,Upon RS,OFFSHORE SCENARIO,YES
4,PROSPECTS,Social Contact \n,Provide contact details,Retail,Pro-actively,OFFSHORE SCENARIO,YES


In [11]:
dfs[7].head()

,CONNECTION TYPE,GROUP,ACTIVITY,SUBACTIVITY,CLIENT TYPE,INTERACTION MODE,SCENARIO,OUTCOME
0,EXISTING CLIENTS,Preparatory administrative activities \n,Provide forms and contractual documents,Account opening contracts including related forms,Retail,Pro-actively,OFFSHORE SCENARIO,YES
1,EXISTING CLIENTS,Preparatory administrative activities \n,Provide forms and contractual documents,Account opening contracts including related forms,Retail,Upon RS,OFFSHORE SCENARIO,YES
2,EXISTING CLIENTS,Preparatory administrative activities \n,Provide forms and contractual documents,Account opening contracts including related forms,Professionals,Pro-actively,OFFSHORE SCENARIO,YES
3,EXISTING CLIENTS,Preparatory administrative activities \n,Provide forms and contractual documents,Account opening contracts including related forms,Professionals,Upon RS,OFFSHORE SCENARIO,YES
4,EXISTING CLIENTS,Preparatory administrative activities \n,Provide forms and contractual documents,Advisory and discretionary mandates including ...,Retail,Pro-actively,OFFSHORE SCENARIO,YES


In [16]:
df = pd.concat(dfs, sort=False)
df.reset_index(drop=True, inplace=True)
df.head()

,CONNECTION TYPE,GROUP,ACTIVITY,CLIENT TYPE,INTERACTION MODE,SCENARIO,OUTCOME,SUBACTIVITY,CONTRACTUAL SETUP
0,PROSPECTS,Social Contact \n,Social interactions\n,Retail,Pro-actively,OFFSHORE SCENARIO,YES,NaN,NaN
1,PROSPECTS,Social Contact \n,Social interactions\n,Retail,Upon RS,OFFSHORE SCENARIO,YES,NaN,NaN
2,PROSPECTS,Social Contact \n,Social interactions\n,Professionals\n,Pro-actively,OFFSHORE SCENARIO,YES,NaN,NaN
3,PROSPECTS,Social Contact \n,Social interactions\n,Professionals\n,Upon RS,OFFSHORE SCENARIO,YES,NaN,NaN
4,PROSPECTS,Social Contact \n,Provide contact details,Retail,Pro-actively,OFFSHORE SCENARIO,YES,NaN,NaN


In [23]:
df.to_json(orient='table')

'{"schema":{"fields":[{"name":"index","type":"integer"},{"name":"CONNECTION TYPE","type":"string"},{"name":"GROUP ","type":"string"},{"name":"ACTIVITY","type":"string"},{"name":"CLIENT TYPE","type":"string"},{"name":"INTERACTION MODE","type":"string"},{"name":"SCENARIO","type":"string"},{"name":"OUTCOME","type":"string"},{"name":"SUBACTIVITY ","type":"string"},{"name":"CONTRACTUAL SETUP","type":"string"}],"primaryKey":["index"],"pandas_version":"0.20.0"},"data":[{"index":0,"CONNECTION TYPE":"PROSPECTS","GROUP ":"Social Contact \\n","ACTIVITY":"Social interactions\\n","CLIENT TYPE":"Retail","INTERACTION MODE":"Pro-actively","SCENARIO":"OFFSHORE SCENARIO","OUTCOME":"YES","SUBACTIVITY ":null,"CONTRACTUAL SETUP":null},{"index":1,"CONNECTION TYPE":"PROSPECTS","GROUP ":"Social Contact \\n","ACTIVITY":"Social interactions\\n","CLIENT TYPE":"Retail","INTERACTION MODE":"Upon RS","SCENARIO":"OFFSHORE SCENARIO","OUTCOME":"YES       ","SUBACTIVITY ":null,"CONTRACTUAL SETUP":null},{"index":2,"CONNE